In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
import csv
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import manifold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR, SVC
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from scipy.spatial.distance import cdist
from sklearn.preprocessing import Imputer
from fancyimpute import SoftImpute

Using TensorFlow backend.
/home/ananth/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Main Flow

In [2]:
#Generate cross correlation file
imp_df = read_csv('/home/ananth/Downloads/tanzania_2014_v8_cleaned.csv')
imp_cols = imp_df.columns.values
# corr = imp_df.corr('spearman')
corr = imp_df.corr('spearman')['crop_sales___output']
corr.to_csv('tanzania_corr.csv')
#Inspect this file manually and see if all features are written correctly and add an extra row on top with the column name:crop_sales___output___tanzania_2014 

In [3]:
imp_df = read_csv('/home/ananth/Downloads/tanzania_2014_v8_cleaned.csv')
cdf = imp_df.loc[imp_df['crop_sales___output'].dropna().index] #___tanzania_2014
# Print missingness values for relevant inputs.
print(100-(cdf.apply(lambda x: x.count(), axis=0)/len(cdf))*100.0)

#Plot variations of input features w.r.t output to pick variables to cluster on.
varies = imp_df.groupby(pd.qcut(imp_df['crop_sales___output'],5,duplicates='drop')).mean()
plt.clf()
varies.plot(x='crop_sales___output', subplots=True,legend=True, figsize=(50,200),kind='bar',fontsize=20)
plt.savefig('variations.pdf')

y4_hhid                                   0.000000
children_education___output              15.535957
crop_sales___output                       0.000000
expenditure___output                      0.000000
has_medical_assistance___output           0.000000
no_food_deficiency___output               0.135685
crop_diversification___policy             4.002714
has_bank_account___policy                 0.000000
has_borrowed___policy                     0.000000
has_hired_workers___policy               29.036635
months_hired_workers___policy             0.000000
number_of_animals_owned___policy         33.175034
number_of_hired_workers___policy         44.979647
number_of_ploughs_owned___policy          0.000000
owns_land_certificate___policy           19.063772
quantity_of_fertilizers_used___policy     0.067843
quantity_of_pesticides_used___policy      0.067843
uses_credit___policy                      0.000000
uses_irrigation___policy                  0.949796
attended_school                

In [18]:
# Reset matplotlib defaults if plots are skewed after generating variations.pdf.
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update(mpl.rcParamsDefault)

In [13]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import re
from scipy import stats
import glmnet_python
from glmnet import glmnet; from glmnetPlot import glmnetPlot 
from glmnetPrint import glmnetPrint; from glmnetCoef import glmnetCoef; from glmnetPredict import glmnetPredict
from cvglmnet import cvglmnet; from cvglmnetCoef import cvglmnetCoef
from cvglmnetPlot import cvglmnetPlot; from cvglmnetPredict import cvglmnetPredict
import math
import statsmodels.api as sm

country = 'tanzania'

def get_classes(output_var, pred):
    max_bins = 3
    _, boundaries = np.histogram(output_var, bins=max_bins)
    classes = np.digitize(pred, bins=boundaries)
    return classes, max_bins

def for_year(var, year):
    return var + '___' + country + '_' + str(year)

def run_regressions(fixed_k, in_name, out_dir, non_policy_inputs, segment_variables, inputs, output, year):
    global table
    global coef_table
    global avg_table
    global coef_map
    # table for regressions and classification
    table = pd.DataFrame()
    avg_table = pd.DataFrame()
    coef_map = {}
    # create table of coefficients
    coef_table = pd.DataFrame()
    
    try:
        os.mkdir(out_dir)
    except:
        print("Dir exists")
    
    ols = linear_model.LinearRegression()
    ridge = linear_model.Ridge(alpha=.5)
    lasso = linear_model.Lasso(alpha = 0.001, max_iter=1e5)
    lars_lasso = linear_model.LassoLars(alpha=.1)
    bayes_ridge = linear_model.BayesianRidge()
    sgd = linear_model.SGDRegressor()
    svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
    svr_lin = SVR(kernel='linear', C=1e3)
    svr_poly = SVR(kernel='poly', C=1e3, degree=2)
    kernel_ridge = KernelRidge(alpha=1.0)

    # pick which regression algos to use
    regression_algorithms = (
#        ('OrdinaryLeastSquares', ols),
#        ('RidgeRegression', ridge),
        ('Lasso', lasso),
#         ('LARS Lasso', lars_lasso),
#         ('BayesianRidgeRegression', bayes_ridge),
#         ('StochasticGradientDescent', sgd),
#         ('SupportVectorRegressionRBF', svr_rbf),
#         ('SupportVectorRegressionLinear', svr_lin),
#         ('SupportVectorRegressionPolynomial', svr_poly),
#         ('KernelRidgeRegression', kernel_ridge)
    )
    
    df = read_csv(in_name)
    df = df.loc[df[output].dropna().index] # drop rows with unobserved income
    df = df.loc[df['weight'].dropna().index]
    df = df.loc[df[output] != 0] # drop zero outputs
    # Transform input
    logged_inputs = ['crop_sales___output', 'expenditure___output', 'crop_diversification___policy', 'number_of_animals_owned___policy', 'number_of_hired_workers___policy', 'quantity_of_fertilizers_used___policy', 'quantity_of_pesticides_used___policy', 'distance_to_road', 'distance_to_market', 'household_size', 'land_surface']
    for inp in logged_inputs:
        df[inp] = df[inp].apply(lambda x: np.log(1+x))

    df['nid']= df.index.tolist()
    
    # Uncomment to add filters based on either gender or location or zero outputs.
    #     df = df.loc[df['household_head_is_male___ethiopia_2015']==0]
    #     filter_var = 'lives_in_amhara___ethiopia_' + str(year)
    #     df = df[df[filter_var]==True]
    #     df = df.loc[df[output] != 0] # drop zero outputs

    # select % of data in test set
    test_split = 0.2
    
    # perform matrix completion. completed is returned as a np array
    # we've discussed not using it, but I left it in because I wasn't able to
    # fit the StandardScaler with a DataFrame that contained NaN's
    imp = Imputer(strategy="mean")
    completed = imp.fit_transform(df)
    
    # reconstruct dataframe with completed matrix
    cols = df.columns.values
    mat = pd.DataFrame(completed,columns=cols)
    
    # Redo the same, but without the transformations, only matrix completion for raw matrix.
    raw_df = read_csv(in_name)
    raw_df = raw_df.loc[raw_df[output].dropna().index]
    raw_df = raw_df.loc[raw_df['weight'].dropna().index]
    raw_df = raw_df.loc[raw_df[output] != 0] # drop zero outputs
    raw_df['nid']= raw_df.index.tolist()
    raw_imp = Imputer(strategy="mean")
    raw_completed = raw_imp.fit_transform(raw_df)
    raw_cols = raw_df.columns.values
    raw_df = pd.DataFrame(raw_completed,columns=raw_cols)
    raw_df['productivity'] = raw_df['crop_sales___output']/raw_df['land_surface']
    raw_df['productivity'] = raw_df['productivity'].apply(lambda x: 0 if x == np.inf else x)

    # z-score the matrix mat used for clustering/regression.
    mat_scaled = StandardScaler()
    mat_scaled.fit(mat)
    mat_sc = mat_scaled.transform(mat)
    mat = pd.DataFrame(mat_sc, columns=mat.columns)
    
    y = mat[output]
    x = mat[inputs]
    
    def update_best_lambda(x_scaled):
        global regression_algorithms
        copy_y = np.array(y, dtype=np.float64)
        print (copy_y)
        fit = cvglmnet(x = x_scaled.copy(), y = copy_y)
        print(fit['lambda_min'])
        regression_algorithms = (('Lasso', linear_model.Lasso(alpha=fit['lambda_min'], max_iter=1e5)))
    
    # Split test/train
    indices = range(len(mat))
    x_train, x_test, y_train, y_test, ind_train, ind_test = \
        train_test_split(x, y, indices, test_size=test_split, random_state=42)
    
    def get_train_test(input_vars):
        x = mat[input_vars].copy()
        x_scaled = StandardScaler()
        x_scaled.fit(x)
        x_sc = x_scaled.transform(x)
        # reconstruct DataFrame
        x = pd.DataFrame(x_sc, columns=x.columns)
        training_x = x.iloc[ind_train, :]
        testing_x = x.iloc[ind_test, :]
        return x_sc, training_x, testing_x
    
    def digitize(output_var, pred):
        from sklearn.preprocessing import label_binarize
        classes, max_bins = get_classes(output_var, pred)
        b_classes = label_binarize(classes, range(max_bins))
        return b_classes
    
    def calc_unsegmented(baseline, x_train, x_test): 
        global table
        global coef_map
        # reg keeps predictions from regressions along with keys
        reg = dict()
        for name, algo in regression_algorithms:
            reg[name] = {}

        # keys and values from test data
        keys_list = []
        y_list = []
        for k, v in y_test.iteritems():
            keys_list.append(k)
            y_list.append(v)

        # run regressions on full dataset
        for name, algo in regression_algorithms:
#             model = algo.fit(x_train,y_train)
#             y_pred = model.predict(x_test)
            model = sm.OLS(y_train, x_train)
            fit = model.fit_regularized(alpha=1e-5, refit=True)
            y_pred = fit.predict(x_test)

            # add predictions to dict
            for i, p in enumerate(y_pred):
                t = reg[name]
                t[keys_list[i]] = p

            try:
                test_c = digitize(y, y_test)
                pred_c = digitize(y, y_pred)
                auc_c = roc_auc_score(test_c, pred_c, average='macro')
            except ValueError:
                auc_c = 0.5
            mse = mean_squared_error(y_test,y_pred)
            scaled_mse = (mse/np.std(y))

            # add row to table
            new_row = pd.DataFrame({'model': name, 'segment': '', 'input': baseline, 'scaled_mse': scaled_mse, 'mse': mse, 'roc_auc': auc_c, 'clustered': False}, index=[0])
            table = table.append(new_row, ignore_index=True)

            # add coefficients to map
#             coef_map[name + '_' + baseline] = model.coef_
            coef_map[name + '_' + baseline] = fit.params
            lower_bounds = []
            upper_bounds = []
            for ci in fit.conf_int():
                lower_bounds += [ci[0]]
                upper_bounds += [ci[1]]
            coef_map[name + '_' + baseline + '_lower_bound'] = lower_bounds
            coef_map[name + '_' + baseline + '_upper_bound'] = upper_bounds
    
    def calc_segmented(segment_variables, baseline, x_train, x_test):
        global table
        global coef_map
        global avg_table
        segment_vars = list(segment_variables.keys())
        
        def add_clusters():
            # elbow method
            sse = []
            seg_data = mat[segment_vars]
            for seg_var in segment_vars:
                # Uncomment if clusters' features need to be weighted.
                # seg_data[seg_var] = seg_data[seg_var]*mat[for_year('weight', year)]
                # Multiply covariance weights so that features of the clusters are oriented towards output.
                seg_data[seg_var] = seg_data[seg_var].apply(lambda x: x*segment_variables[seg_var])
            for k in range(1,9):
                kmeans = KMeans(n_clusters=k).fit(seg_data)
                labels = kmeans.labels_
                sse.append(sum(np.min(cdist(seg_data, kmeans.cluster_centers_, 'euclidean'), axis=1)) / seg_data.shape[0])

            # K-means elbow calculation
            plt.clf()
            plt.plot(range(1,9), sse)
            plt.xlabel('k')
            plt.ylabel('Sum of squared error')
            plt.savefig(os.path.join(out_dir, 'elbow.png'))
            print(sse)
            min_k = sse.index(min(sse))
            print (min_k)
            
            # K-means fixed K calculation.
            min_k = fixed_k
            kmeans = KMeans(n_clusters=min_k).fit(seg_data)

            labels = kmeans.labels_
            mat['cluster'] = labels
            # Sort cluster labels in order of mean of output within cluster.
            means = []
            for i in np.unique(labels):
                sel_mat = mat[mat['cluster'] == i]
                raw_clus = raw_df.loc[sel_mat.index]
                values = list(raw_clus[output].as_matrix())
                # Weighted average can be done if the weight column exists.
                weights = list(raw_clus['weight'].as_matrix())
#                 print (values)
#                 print(weights)
                average = np.average(values, weights=weights)
                means.append(average)
            sorted_ids = [i[0] for i in sorted(enumerate(means), key=lambda x:x[1])]
            print(sorted_ids)
            mat['cluster'] = mat['cluster'].apply(lambda x: sorted_ids.index(x))

            # Output ids of households, their cluster number, variables on which cluster is done along with lat/long if exists
            # select_variables += ['latitude___ethiopia_2015', 'longitude___ethiopia_2015']
            # Note (Sam): This is where the file I give you with Ids, cluster numbers is written.
            # The baseline = relevant variables
            select_variables = [output, 'y4_hhid'] + segment_vars
            select_variables = [t.replace('norm', 'raw') for t in select_variables]
            all_output = pd.concat([mat['cluster'], raw_df[select_variables]], 1)
            all_output.to_csv(os.path.join(out_dir,'clus_' + baseline + '_' + output + '.csv'))
            return min_k
            
        
        # Add segments based on median.
        def add_segments():
            median_segments = {}
            for seg_var in segment_vars:
                #binary
                if len(np.unique(mat[seg_var])) <= 3:
                    median_segments[seg_var] = 0
                else:
                    median_segments[seg_var] = np.median(mat[seg_var])
            mat['cluster'] = 0
            for seg_var in segment_vars:
                mat['cluster'] = 2*mat['cluster'] + [int(x) for x in mat[seg_var] > median_segments[seg_var]]
            #mat[['cluster'] + segment_vars].to_csv(os.path.join(out_dir,'segment_' + ','.join(segment_vars) + '_' + baseline + '_' + output + '.csv'))
            return int(math.pow(2, len(segment_vars)))

        # Segments based solely on location.
        def add_location_segments():
            locations = ['afar', 'amhara', 'benishangul_gumuz', 'dire_dawa', 'gambella', 'harari', 'oromiya', 'snnp', 'somalie', 'tigray']
            i = 0
            mat['cluster'] = 0
            for l in sorted(locations):
                loc_feature = 'lives_in_' + l + '___ethiopia_' + str(year)
                loc_val = mat[loc_feature].apply(lambda x: 0 if x < 0 else 1)
                mat['cluster'] = mat['cluster'] + (i*loc_val)
                i += 1
            return len(locations) + 1
            
        def _run(max_clusters, method_name):
            global table
            global avg_table
            global coef_map
            # reg_clus keeps predictions from clustered regressions along with keys
            reg_clus = dict()

            for name, algo in regression_algorithms:
                reg_clus[name] = {}

            # need new dataframes with only training and test rows.
            # we use this when looping through clusters
            train_mat = mat.loc[ind_train]
            test_mat = mat.loc[ind_test]
            train_size = len(train_mat)
            
            series = {}
            series[output] = []
            for seg in segment_vars:
                series[seg] = []
            series = pd.DataFrame()
            row = {}
            raw_cols = raw_df.columns.values
            raw_reg = re.compile('^((?!norm).)*$') #+ str(year) +
            # avg_variables is set of all variables whose mean, 25%ile, 75%ile, stddev, stderr stats are written to *_avg file.
            avg_variables = list(filter(raw_reg.search, raw_cols))
            for i in range(max_clusters):
                train_clus = x_train.loc[train_mat['cluster'] == i]
                train_y = y_train.loc[train_mat['cluster'] == i]
                test_clus = x_test.loc[test_mat['cluster'] == i]
                test_y = y_test.loc[test_mat['cluster'] == i]
                sel_mat = mat[mat['cluster'] == i]
                raw_clus = raw_df.loc[sel_mat.index]

                for seg in avg_variables:
                    values = raw_clus[seg].as_matrix()
                    # Weighted average if weight column exists.
                    weights = raw_clus['weight'].as_matrix()
                    average = np.average(values, weights=weights)
                    row['mean_' + seg] = average
#                     print (average)
                    variance = np.average((values-average)**2, weights=weights)
                    row['stddev_' + seg] = math.sqrt(variance)
                    row['stderr_' + seg] = math.sqrt(variance)/math.sqrt(len(values))
                    row['25ile_' + seg] = np.percentile(values, 25)
                    row['75ile_' + seg] = np.percentile(values, 75)
                
                row['index'] = i
                row['size'] = len(raw_clus)
                new_row = pd.DataFrame(row, index=[0])
                series = series.append(new_row, ignore_index=True)
                avg_table = avg_table.append(new_row, ignore_index=True)
                cluster_percent = (len(train_clus)*100.0)/train_size
                if train_clus.empty or test_clus.empty:
                    continue

                keys_list = []
                y_list = []
                for k, v in test_y.iteritems():
                    keys_list.append(k)
                    y_list.append(v)

                # Regress per cluster
                for name, algo in regression_algorithms:  
#                     model = algo.fit(train_clus,train_y)
#                     y_pred = model.predict(test_clus)
                    model = sm.OLS(train_y, train_clus)
                    fit = model.fit_regularized(alpha=1e-5, refit=True)
                    y_pred = fit.predict(test_clus)

                    for a, b in enumerate(y_pred):
                        t = reg_clus[name]
                        t[keys_list[a]] = b

#                     coef_map[name + '_' + method_name + '_' + ','.join(segment_vars) + '_' + str(i)] = model.coef_
                    coef_map[name + '_' + method_name + '_' + ','.join(segment_vars) + '_' + str(i)] = fit.params
                    lower_bounds = []
                    upper_bounds = []
                    for ci in fit.conf_int():
                        lower_bounds += [ci[0]]
                        upper_bounds += [ci[1]]
                    coef_map[name + '_' + method_name + '_' + str(i) + '_lower_bound'] = lower_bounds
                    coef_map[name + '_' + method_name + '_' + str(i) + '_upper_bound'] = upper_bounds

            # plot sorted correlation
            sorted_series = series.sort_values(['mean_' + output])
#             print (sorted_series)
            for seg in avg_variables:
                plt.clf()
                plt.plot(sorted_series['mean_' + output].as_matrix(), sorted_series['mean_'+seg].as_matrix(), marker='o')
                plt.xlabel('Average ' + output.replace('___output___' + country + '_' + str(year), '') + ' output')
                plt.ylabel('Average ' + seg.replace('___policy___'  + country + '_' +str(year), '').replace('_',' '))
                plt.savefig(os.path.join(out_dir, 'plot_' + seg + '_' + output + '.pdf'))
                
            # add mse's to table
            keys = sorted(y_test.keys())
            for name, algo in regression_algorithms:
                sort_t = []
                sort_p = []

                for key in keys:
                    if key not in y_test or key not in reg_clus[name]:
                        continue
                    sort_t.append(reg_clus[name][key])
                    sort_p.append(y_test[key])

                try:
                    test_c = digitize(y, sort_t)
                    pred_c = digitize(y, sort_p)
                    auc_c = roc_auc_score(test_c, pred_c, average='macro')
                except ValueError:
                    auc_c = 0.5
                mse = mean_squared_error(sort_t,sort_p)
                scaled_mse = (mse/np.std(y))
                new_row = pd.DataFrame({'model': name, 'segment': ','.join(segment_vars), 'input': baseline, 'scaled_mse': scaled_mse, 'mse': mse, 'roc_auc': auc_c, 'clustered': True, 'method': method_name}, index=[0])
                table = table.append(new_row, ignore_index=True)
                
    
        ##### Run grouped regressions
        #_run(add_location_segments(), 'segmented')
        if (len(segment_vars) > 1):
            _run(add_clusters(), 'clustered')
        #_run(add_segments(), 'segmented')
    
    def run_with_inputs(input_vars, name):
        # map to be used in tracking coefficients
        global coef_map
        global coef_table
        global x_train
        global x_test
        coef_map = {}
        x_scaled, x_train, x_test = get_train_test(input_vars)
        # Update Lasso Lambda using GLMNET.
        calc_unsegmented(name, x_train, x_test)
        update_best_lambda(x_scaled)
        calc_segmented(segment_variables, name, x_train, x_test)
        
        for k,v in sorted(coef_map.items()):
            kvp = dict()
            kvp['model'] = k
            kvp['inputs'] = name

            for val,invar in zip(v,input_vars):
                kvp[invar] = val

            new_row = pd.DataFrame(kvp, index=[0])
            coef_table = coef_table.append(new_row, ignore_index=True)
    
    # Baseline 1
    input_vars = inputs + non_policy_inputs
#     run_with_inputs(input_vars, 'All variables')
    # Baseline 2 - only policy variables that have high correlations.
    # Note(Sam): Modifying this regex will change variables to regress on.
    imp_vars = re.compile('^.*(?=plough|has_hired|irrigation|animals|borrowed).*$')
    input_vars = list(filter(imp_vars.search, input_vars))
    run_with_inputs(input_vars, 'Highly correlated policy and non-policy variables')
    # Baseline 3 with only policy variables
#     run_with_inputs(inputs, 'Policy variables')    

    # save coefficient and output tablesdrop
    coef_table.to_csv(os.path.join(out_dir,'coef_' + output + '.csv'))
    table.to_csv(os.path.join(out_dir,output + '.csv'))
    avg_table.to_csv(os.path.join(out_dir,output + '_avg' + '.csv'))

In [6]:
# Functions to just populate clusters across years.

def get_segments(df, output):
    df_t = df.loc[df[output].dropna().index]
    df['segment_' + output], _ = get_classes(df_t[output], df[output])
    return df

def complete(df):
    imp = Imputer(strategy="mean")
    completed = imp.fit_transform(df)
    cols = df.columns.values
    mat = pd.DataFrame(completed,columns=cols)
    mat_scaled = StandardScaler()
    mat_scaled.fit(mat)
    mat_sc = mat_scaled.transform(mat)
    mat = pd.DataFrame(mat_sc, columns=mat.columns)
    return mat

# Although this repeats calculation done above with clustering/regression, it allows us to add columns per year
# in the same dataframe.
def get_clusters(mat, output, segment_vars):
    segment_vars = list(segment_vars.keys())
    seg_data = mat[segment_vars]
    min_k = 4
    kmeans = KMeans(n_clusters=min_k).fit(seg_data)
    labels = kmeans.labels_
    means = []
    for i in np.unique(labels):
        df_clus = mat.loc[labels == i]
        means.append(np.mean(df_clus[output].as_matrix()))
    sorted_ids = [i[0] for i in sorted(enumerate(means), key=lambda x:x[1])]
    mat['segment_'+ output] = labels
    mat['segment_'+ output] = mat['segment_'+ output].apply(lambda x: sorted_ids[x])
    return mat

In [14]:
import re

filename = '/home/ananth/Downloads/tanzania_2014_v8_cleaned.csv'
df = pd.read_csv(filename)
country = 'tanzania'

# change to true if you want to use all input fields
def get_vars(year):
    base_suffix = ''#'___'+country+'_{0}'.format(year)
    year_vars = df.filter(regex='.*{0}'.format(base_suffix)).columns.values
    # If we need to filter out by variables or year.
    # non_raw_reg = re.compile('^((?!gender|damaged|bank|price|irrigation|diversification).)*$')
    # year_vars = list(filter(non_raw_reg.search, year_vars))
    out_reg = re.compile('.*' + base_output + '___output.*$')
    outputs = list(filter(out_reg.search, year_vars))
    policy_reg = re.compile('(.*___policy.*)$')
    policy_inputs = list(filter(policy_reg.search, year_vars))
    non_policy_reg = re.compile('^((?!policy|output|weight).)*$')
    non_policy_inputs = list(filter(non_policy_reg.search, year_vars))
    
    return outputs, policy_inputs, non_policy_inputs

# Modify this based on the years in the dataset.
# years = [2011, 2013, 2015]
years = [2014]

# When true, df will contain columns for clusters across years, which then can be used to calculate
# evidence of change across years and agreement numbers.
populate_across_year_clusters = True

for base_output in ['crop_sales',]:
    # Choose variables to segment on based on correlation file.
    ccs = pd.read_csv(country + '_corr.csv')
    output = base_output + '___output' + '___'+country+'_2014'
    ccs[output] = ccs[output].apply(lambda x: abs(x))
    # In some cases, we choose based on stability across variations in clusters after manual inspection.
    # Note(Sam): Modifying this regex will change variables to cluster on.
    select = ccs['Unnamed: 0'].str.contains('^.*(?=animals|household_size|has_hired|land_surface|widowed).*$') # |quantity_of_pesticides,crop_diversification, separated
    ccs = ccs[select]
    ccs = ccs.sort_values(output, ascending=False)
    num_vars=8
    best_vars = ccs[['Unnamed: 0', output]][:num_vars].as_matrix()
    segment_variables = {}
    seg_vars = []
    for i in best_vars:
        name = i[0]
        segment_variables[name] = i[1]

    print(segment_variables)

    raw_df = df.copy()
    df = complete(df)
    for year in years:
        outputs, policy_inputs, non_policy_inputs = get_vars(year)
        for output in outputs:
            if populate_across_year_clusters:
                df = get_clusters(df, output, segment_variables)
            run_regressions(fixed_k=4, in_name=filename, out_dir='./tanzania_v6_7_' + base_output, non_policy_inputs=non_policy_inputs, segment_variables=segment_variables, inputs=policy_inputs, output=output, year=year)

{'land_surface': 0.35053444487641, 'number_of_animals_owned___policy': 0.28171114112379303, 'has_hired_workers___policy': 0.19052291722913, 'household_size': 0.17627112215002, 'household_head_is_widowed': 0.10803100253121302}
Dir exists
[ 0.07582827  0.48055669 -0.90791973 ..., -2.50892367 -1.4343748
 -1.65375061]
[ 0.00321071]

/home/ananth/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



[0.48209705328214764, 0.38923715116180546, 0.35584060303805398, 0.32792634240295437, 0.30445543907817291, 0.29279452166475445, 0.27706305580272506, 0.26784019108643575]
7
[3, 0, 1, 2]


In [16]:
# Run this once to avoid overwriting df in the code below.
df_all = df

In [ ]:
imp_df = read_csv('/home/ananth/Downloads/tanzania_2014_v8_cleaned.csv')
imp_cols = imp_df.columns.values
imp_feats = set([])
for y in [2011, 2013, 2015]:
    y_reg = re.compile('.*___.*' + str(y) + '$')
    y_cols = set(filter(y_reg.search, imp_cols))
    y_cols = set([t.replace('___tanzania_' + str(y), '') for t in y_cols])
    if len(imp_feats) == 0:
        imp_feats = y_cols
    else:
        imp_feats = imp_feats.intersection(y_cols)
imp_feats

In [17]:
# Compute evidence of movement across years, the lift due to movement in relevant inputs.
df = df_all
series = pd.DataFrame()
for imp_feat in imp_feats:
    for output in ['segment_crop_sales___output']:
        years = ['2011', '2013', '2015']
        raw_output = 'crop_sales___output'
        coef= {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1}
        for [y1, y2] in [['2011', '2013'], ['2013', '2015']]:
            #try:
            z1 = for_year(output, y1)
            z2 = for_year(output, y2)
            r1 = for_year(raw_output, y1)
            r2 = for_year(raw_output, y2)
            f1 = for_year(imp_feat,y1)
            f2 = for_year(imp_feat,y2)
            df[output + '_change' + y1] = (df[z1]!=df[z2])
            df[output + '_increase' + y1] = (df[z1]<df[z2])
            df[output + '_decrease' + y1] = (df[z1]>df[z2])
            expected = df[z1].apply(lambda x: coef[x])
            df[imp_feat+'_increase'+y1] = (imp_df[f1]<imp_df[f2])
            df[output + '_inversion' + y1] = (df[z2]-df[z1])*(df[f2]-df[f1])*expected
            df[output + '_change_value' + y1] = (imp_df[r2]-imp_df[r1])

        for per in [50,55,60,70,75,80,85,90,95]:
            for seg in range(4):
                exp_y_i = []
                exp_y = []
                exp = []
                exp_i = []
                for [y1, y2] in [['2011', '2013'], ['2013', '2015']]:
                    year = y1
                    weight = for_year('weight', year)
                    seg_y = for_year(output, year)
                    df_seg = df[df[seg_y]==seg]
                    df_seg = df_seg.loc[df_seg[output + '_change_value' + year].dropna().index]
                    thres = np.percentile(df_seg[output + '_change_value' + year].dropna(), per)
                    high_df = df_seg[df_seg[output + '_change_value' + year].apply(lambda x : x >= thres)]
                    exp_y_i.append(len(high_df[(high_df[imp_feat + '_increase'+year]==True)]))
                    exp_y.append(len(high_df))
                    exp.append(len(df_seg))
                    exp_i.append(len(df_seg[df_seg[imp_feat + '_increase'+year]==True]))
                    avg_y_i = np.mean(high_df[(high_df[imp_feat + '_increase'+year]==True)][output + '_change_value' + year])
                    avg_y = np.mean(high_df[output + '_change_value' + year].dropna())
                    avg = np.median(df[output + '_change_value' + year].dropna())
                row = {}
                row['threshold'] = per
                row['input'] = imp_feat
                row['cluster'] = seg
                row['movement overall'] = (sum(exp_y)/sum(exp))*100.0
                row['movement conditioned'] = (sum(exp_y_i)/sum(exp_i))*100.0
                row['movement lift'] = (sum(exp_y_i)/sum(exp_i))/(sum(exp_y)/sum(exp))
                new_row = pd.DataFrame(row, index=[0])
                series = series.append(new_row, ignore_index=True)

series.to_csv('./threshold-lift-tanzania.csv')

NameError: name 'imp_feats' is not defined

In [ ]:
# Inspect best cross correlated variables
# Might vary for Tanzania/Uganda.
ccs = pd.read_csv('cross_correls_ethiopia_2015_v22.csv')
policy = ccs['Unnamed: 0'].apply(lambda x: x.find("___policy") != -1)
no_lives = ccs['Unnamed: 0'].apply(lambda x: x.find("lives_in") == -1)
no_lat = ccs['Unnamed: 0'].apply(lambda x: x.find("latitude") == -1)
no_dist = ccs['Unnamed: 0'].apply(lambda x: x.find("distance") == -1)
no_lon = ccs['Unnamed: 0'].apply(lambda x: x.find("longitude") == -1)
no_equ = ccs['Unnamed: 0'].apply(lambda x: x.find("equal") == -1)
ccs = ccs[no_lives & no_lat & no_lon & no_equ & no_dist]
cols = ccs.columns.values
best_var_series = {}
for output in cols:
    if output == 'Unnamed: 0':
        continue
    ccs[output] = ccs[output].apply(lambda x: abs(x))
    ccs = ccs.sort_values(output, ascending=False)
    best_vars = ccs[['Unnamed: 0', output]][:15].as_matrix()
    best_var_series[output] = best_vars
best_var_series['crop_sales___output___ethiopia_2015']

In [95]:
# Print highly correlated policy recommendations.
collinear = read_csv('./policy_collinear.csv')
cols = collinear.columns.values
for c in cols:
    if c == 'Unnamed: 0' or not(c.find('hired')!=-1 or c.find('oxen')!=-1 or c.find('axe')!=-1 or c.find('water')!=-1 or c.find('extension')!=-1 or c.find('fertilizer')!=-1):
        continue
    print(c)
    print(collinear.nlargest(5, c)[['Unnamed: 0', c]].as_matrix()[1:])

number_of_axe_owned___policy___ethiopia_2015
[['number_of_pick_axe_owned___policy___ethiopia_2015' 0.3153059718665343]
 ['crop_diversification___policy___ethiopia_2015' 0.14174075630319244]
 ['number_of_oxen_owned___policy___ethiopia_2015' 0.10308466772245027]
 ['increase_in_price_of_inputs___policy___ethiopia_2015'
  0.09004134147752284]]
number_of_pick_axe_owned___policy___ethiopia_2015
[['number_of_axe_owned___policy___ethiopia_2015' 0.3153059718665343]
 ['crop_diversification___policy___ethiopia_2015' 0.1839787891236533]
 ['number_of_plough_owned___policy___ethiopia_2015' 0.12247590046462667]
 ['number_of_oxen_owned___policy___ethiopia_2015' 0.10963989145180954]]
number_of_hired_workers___policy___ethiopia_2015
[['uses_extension_program___policy___ethiopia_2015' 0.11621771224868127]
 ['quantity_of_improved_seeds_used___policy___ethiopia_2015'
  0.114399538356677]
 ['quantity_of_chemical_fertilizers_used___policy___ethiopia_2015'
  0.11266978387196736]
 ['number_of_water_storage_pit

In [507]:
# Write input cross-correlation rank scores to csv file.
feats = list(imp_feats)
feats = [for_year(f, 2015) for f in feats]
print(feats)
imp_vars = re.compile('^.*(?=improved_seeds|water_storage|saved|extension|plough|oxen|hired_workers|fertilizer|health_issues|sickle|axe|credit).*$')
input_vars = list(filter(imp_vars.search, feats))
collinear = imp_df[feats].corr('spearman')
collinear.to_csv('./policy_collinear.csv')

['owns_land_certificate___policy___ethiopia_2015', 'price_rise_of_food_item___policy___ethiopia_2015', 'percentage_of_damaged_crop___policy___ethiopia_2015', 'number_of_axe_owned___policy___ethiopia_2015', 'number_of_pick_axe_owned___policy___ethiopia_2015', 'number_of_hired_workers___policy___ethiopia_2015', 'number_of_water_storage_pit_owned___policy___ethiopia_2015', 'uses_extension_program___policy___ethiopia_2015', 'number_of_oxen_owned___policy___ethiopia_2015', 'illness_of_household_member___policy___ethiopia_2015', 'has_health_issues___policy___ethiopia_2015', 'amount_of_assistance_received___policy___ethiopia_2015', 'number_of_plough_owned___policy___ethiopia_2015', 'increase_in_price_of_inputs___policy___ethiopia_2015', 'crop_diversification___policy___ethiopia_2015', 'prevent_damage___policy___ethiopia_2015', 'number_of_sickle_owned___policy___ethiopia_2015', 'has_borrowed___policy___ethiopia_2015', 'uses_credit___policy___ethiopia_2015', 'quantity_of_chemical_fertilizers_us

In [532]:
#Inspect per area % population and average output.
locations = ['afar', 'amhara', 'benishangul_gumuz', 'dire_dawa', 'gambella', 'harari', 'oromiya', 'snnp', 'somalie', 'tigray']
imp_df = imp_df.loc[imp_df[output].dropna().index]
for l in sorted(locations):
    loc_feature = 'lives_in_' + l + '___ethiopia_' + str(2015)
    df_loc = imp_df.loc[imp_df[loc_feature]==1]
    output = 'crop_sales___output___ethiopia_2015'  
    print (l, sum(df_loc['weight___ethiopia_2015'])/sum(imp_df['weight___ethiopia_2015'])*100.0, np.average(df_loc[output], weights=df_loc['weight___ethiopia_2015']))

afar 0.0398055198279 2129.3261381
amhara 25.8150368658 1155.75359963
benishangul_gumuz 1.59543145271 2725.0879217
dire_dawa 0.174135908025 1150.03649518
gambella 0.31624091125 2110.87379671
harari 0.180543438023 8034.89341087
oromiya 41.046503324 1767.39513676
snnp 26.6685963573 1036.59091902
somalie 0.409996338586 1545.2165131
tigray 3.75370988447 1898.11291005


In [601]:
# To determine thresholds for change between clusters over years, some stats on what quantile is the difference
# between mean of outputs of clusters.
np.percentile(imp_df['crop_sales___output___ethiopia_2015'].dropna(), [25,50,75])
np.std(imp_df['crop_sales___output___ethiopia_2015'].dropna())
from scipy.stats import percentileofscore
x = [568.191517737871, 245.304134630863, 902.739225809435]
for i in x:
    print(percentileofscore(df[output + '_change_value' + year].dropna(), i))
# 75%ile overall
np.percentile(df[output + '_change_value' + year].dropna(), 75)

76.4168190128
67.0932358318
82.449725777


489.09268211400331